In [1]:
# NN 6 outputs and status of swapping has to be decided on the basis of experiment 2
model_path = 'saved_model/Multi_leak/experiment3/withswap/sameloss/'
project_name='Multi_leak_experiment3_MTL_withSwap_sameloss'

In [2]:
# Experiment 2 - 1 output layer with 1 loss function - mse. and do hyper parameter tuning.
from utils.data_preprocess import load_data, load_single_leakage_model_data
from utils.module import model_eval, hyper_model, model_comparison, linear_regression, numpy_to_tensor, benchmark_linear_model
import itertools
import pandas as pd 
import yaml
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Model
from keras.layers import Dense, Input
import keras_tuner as kt
from tensorflow import keras
from keras import layers
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from kerastuner import HyperModel, Hyperband
from keras import backend as backend

2.12.1
# GPUs Available:  1


In [3]:
single_leakage, two_leakage = load_data()
# two_leakage["leak_1"] = 1
two_leakage["leak_2"] = 1

# single_leakage["leak_1"] = 1
single_leakage["leak_2"] = 0

data = pd.concat([single_leakage, two_leakage], axis=0)
# use the idea of masking instead
data['x2'] = data['x2'].replace(np.nan, 8024)
data['y2'] = data['y2'].replace(np.nan, 2616.5)

data = data.drop(columns=['mfc6_residual',
       'mfc7_residual', 'mfc8_residual', 'mfc9_residual', 'mfc10_residual',
       'mfc1_residual', 'mfc2_residual', 'mfc3_residual', 'mfc4_residual',
       'mfc5_residual','total flow rate'
       ])

y = data[['x1', 'y1', 'x2', 'y2', 'leak_2']]
x = data.drop(['x1', 'y1', 'x2', 'y2', 'leak_2'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.15, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=1) 


# sort x1, y1 and x2, y2. coordinates with lowest x will take the position of x1
def coords_swap(y1):
    s = y1['x2'] < y1['x1']
    y1.loc[s, ['x1','x2']] = y1.loc[s, ['x2','x1']].values
    y1.loc[s, ['y1','y2']] = y1.loc[s, ['y2','y1']].values
    return y1

y1_train = y_train[['x1', 'y1', 'x2', 'y2']]
y2_train = y_train[[ 'leak_2']]
y1_test = y_test[['x1', 'y1', 'x2', 'y2']]
y2_test = y_test[[ 'leak_2']]
y1_val = y_val[['x1', 'y1', 'x2', 'y2']]
y2_val = y_val[[ 'leak_2']]


# y1_test.to_csv('check1.csv')

y1_data = [y1_train, y1_val, y1_test]
y1_data_types = ['y1_train', 'y1_val', 'y1_test']
for y1_data_types, y1 in zip(y1_data_types, y1_data):
    y1_data_types = coords_swap(y1)

# print()

y1_columns = y1_train.columns
y2_columns = y2_train.columns
X_columns = X_train.columns

# y1_test.to_csv('check2.csv')

scaler_coords1 = StandardScaler()
y1_train = scaler_coords1.fit_transform(y1_train)
y1_test = scaler_coords1.transform(y1_test)
y1_val = scaler_coords1.transform(y1_val)

y1_train = pd.DataFrame(y1_train, columns=y1_columns)
y1_test = pd.DataFrame(y1_test, columns=y1_columns)
y1_val = pd.DataFrame(y1_val, columns=y1_columns)

y2_train = y2_train.reset_index().drop(columns='sample_number')
y2_val = y2_val.reset_index().drop(columns='sample_number')
y2_test = y2_test.reset_index().drop(columns='sample_number')

y_train_all = pd.concat([y1_train, y2_train], axis=1)
y_test_all = pd.concat([y1_test, y2_test], axis=1)
y_val_all = pd.concat([y1_val, y2_val], axis=1)

scaler_flows = StandardScaler()
X_train = scaler_flows.fit_transform(X_train)
X_test = scaler_flows.transform(X_test)
X_val = scaler_flows.transform(X_val)

X_train = pd.DataFrame(X_train, columns=X_columns)
X_test = pd.DataFrame(X_test, columns=X_columns)
X_val = pd.DataFrame(X_val, columns=X_columns)

In [4]:
X_train_np, y1_train_np, y2_train_np = X_train.values, y1_train.values, y2_train.values
X_val_np, y1_val_np, y2_val_np = X_val.values, y1_val.values, y2_val.values
X_test_np, y1_test_np, y2_test_np = X_test.values, y1_test.values, y2_test.values

# Create TensorFlow datasets from NumPy arrays.
batch_size = 32
buffer_size = len(X_train)  # Set the buffer size to the number of training examples for full shuffling.

train_dataset = tf.data.Dataset.from_tensor_slices((X_train_np, y1_train_np, y2_train_np ))
val_dataset = tf.data.Dataset.from_tensor_slices((X_val_np, y1_val_np, y2_val_np))
test_dataset = tf.data.Dataset.from_tensor_slices((X_test_np, y1_test_np, y2_test_np))

# Shuffle, batch, and prefetch the training dataset.
train_dataset_org = train_dataset.batch(batch_size)
train_dataset = train_dataset.shuffle(buffer_size).batch(batch_size).prefetch(tf.data.AUTOTUNE)

# Batch the validation and test datasets.
val_dataset = val_dataset.batch(batch_size)
test_dataset = test_dataset.batch(batch_size)

In [5]:
# y_train = {
#     "y1" : y1_train,
#     "y2" : y2_train
# }

# y_val = {
#     "y1" : y1_val,
#     "y2" : y2_val
# }

# y_test = {
#     "y1" : y1_test,
#     "y2" : y2_test
# }

losses = {
	"y1": "mse",
	"y2": 'mse'
    # "y2" : 'mse'
    }

metrics = {
    "y1": 'mae',
    "y2": 'mae'
    }

In [6]:
# Experiment 2 with Nadam

EPOCHS = 1000
# Define the multi-task HyperModel
class MultiTaskHyperModel(HyperModel):

    def build(self, hp):
        backend.clear_session()
        inputs = keras.Input(shape=(10,))
        shared_layer = inputs
        for i in range(hp.Int('num_layers', 1, 15)):
            shared_layer = layers.Dense(
                units=hp.Int("units_" + str(i), min_value=32, max_value=512, step=32),
                # activation=hp.Choice("activation", ["relu", 'elu']),
                activation = 'relu',
                # add elu
                kernel_initializer='he_uniform'
            )(shared_layer)


        task_layer1 = shared_layer
        for j in range(hp.Int(f'task_{i}_num_layers', 0, 10)):
            task_layer1 = layers.Dense(units=hp.Int(f'task_{i}_layer_{j}_neurons', min_value=8, max_value=128, step=8), 
                                       activation='relu',
                                       kernel_initializer='he_uniform')(task_layer1)
        y1 = layers.Dense(4, name='y1', activation = 'linear', kernel_initializer='he_uniform')(task_layer1)

        task_layer2 = shared_layer
        for j in range(hp.Int(f'task_{i}_num_layers', 0, 10)):
            task_layer2 = layers.Dense(units=hp.Int(f'task_{i}_layer_{j}_neurons', min_value=2, max_value=64, step=2),
                                        activation='relu',
                                        kernel_initializer='he_uniform')(task_layer2)
        y2 = layers.Dense(1, name='y2', activation = 'sigmoid', kernel_initializer='he_uniform')(task_layer2)

        outputs = [y1, y2]

        # loss1_weight = hp.Float("loss1_weight", min_value=1e-4, max_value=1, sampling="log")
        # loss1_weight = hp.Float("loss2_weight", min_value=1e-4, max_value=1, sampling="log")
        
        # loss1_weight = hp.Choice('loss1_weight', values=[0.6, 0.7, 0.8, 0.9])
        # loss2_weight = hp.Choice('loss2_weight', values=[0.00005, 0.2, 0.4, 0.6, 0.8, 1.0])
        # to stop the model from reducing both these weightage to very low and hence reducing the total loss
        # loss2_weight = 1 - loss1_weight

        lossWeights = {"y1": 0.9, "y2": 0.1}

        model = keras.Model(inputs=inputs, outputs=outputs)
        learning_rate = hp.Float("lr", min_value=1e-4, max_value=1e-1, sampling="log")

        # model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        #             loss="mse",  metrics='mae')
        
        model.compile(optimizer=tf.keras.optimizers.Nadam(learning_rate=learning_rate),
                    loss=losses, 
                    loss_weights=lossWeights,
                    metrics = metrics)
        return model

# Create the multi-task HyperModel
multi_task_hypermodel = MultiTaskHyperModel()

# Define the Hyperband tuner
tuner = Hyperband(
    multi_task_hypermodel,
    objective =  kt.Objective("val_loss", direction="min"),
    max_epochs=EPOCHS+100,
    factor=2,
    directory="../../tensorflow_log_files/studienarbeit/",
    project_name=project_name,
    seed=0,   
)

tuner.search_space_summary()
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Perform hyperparameter search
tuner.search(train_dataset.map(lambda x, y1, y2: (x, {'y1': y1, 'y2': y2})),
            #  X_train, y_train, 
            #  validation_data = (X_val, y_val), 
            validation_data = val_dataset.map(lambda x, y1, y2: (x, {'y1': y1, 'y2': y2})), 
             verbose = 1, callbacks=[stop_early],
             epochs=EPOCHS, shuffle = True)

best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print("Best Hyperparameters:", best_hps)

# Build the best model with the best hyperparameters
best_model = tuner.hypermodel.build(best_hps)

# Train the best model on the full dataset
history = best_model.fit(train_dataset.map(lambda x, y1, y2: (x, {'y1': y1, 'y2': y2})),
            #  X_train, y_train, 
            #  validation_data = (X_val, y_val), 
            validation_data = val_dataset.map(lambda x, y1, y2: (x, {'y1': y1, 'y2': y2})),  
            callbacks=[stop_early],
            verbose = 1, epochs=EPOCHS, shuffle = True)

print(f"""
    The hyperparameter search is complete. The optimal learning rate for the optimizer
    is {best_model.optimizer.lr.numpy()}.
    """)

Trial 613 Complete [00h 00m 13s]
val_loss: 0.8743064401430658

Best val_loss So Far: 0.0936473066353321
Total elapsed time: 00h 12m 52s

Search: Running Trial #614

Value             |Best Value So Far |Hyperparameter
14                |3                 |num_layers
288               |480               |units_0
10                |5                 |task_0_num_layers
0.0007071         |0.002596          |lr
96                |480               |units_1
160               |288               |units_2
320               |128               |units_3
256               |288               |units_4
384               |128               |units_5
32                |480               |units_6
224               |448               |units_7
320               |256               |units_8
288               |448               |units_9
320               |320               |units_10
128               |320               |units_11
10                |5                 |task_11_num_layers
6                 |1     

In [ ]:
# best_model.save(model_path)
best_model = tf.keras.models.load_model(model_path)
best_model.summary()

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
y_predictions_train = best_model.predict(train_dataset_org.map(lambda x, y1: (x, {'y1': y1})))
# print("train", "{:10.4f}".format(mean_squared_error(y_train, y_predictions, squared=True)))
y_predictions_val = best_model.predict(val_dataset.map(lambda x, y1: (x, {'y1': y1})))
# print("val", "{:10.4f}".format(mean_squared_error(y_val, y_predictions, squared=True)))
y_predictions = best_model.predict(test_dataset.map(lambda x, y1: (x, {'y1': y1})))

y_predictions_train[:,-2][np.abs(y_predictions_train[:,-2]) < 0.5] = 0
y_predictions_train[:,-2][np.abs(y_predictions_train[:,-2]) > 0.5] = 1
y_predictions_train[:,-1][np.abs(y_predictions_train[:,-1]) < 0.5] = 0
y_predictions_train[:,-1][np.abs(y_predictions_train[:,-1]) > 0.5] = 1

y_predictions_val[:,-2][np.abs(y_predictions_val[:,-2]) < 0.5] = 0
y_predictions_val[:,-2][np.abs(y_predictions_val[:,-2]) > 0.5] = 1
y_predictions_val[:,-1][np.abs(y_predictions_val[:,-1]) < 0.5] = 0
y_predictions_val[:,-1][np.abs(y_predictions_val[:,-1]) > 0.5] = 1

y_predictions[:,-2][np.abs(y_predictions[:,-2]) < 0.5] = 0
y_predictions[:,-2][np.abs(y_predictions[:,-2]) > 0.5] = 1
y_predictions[:,-1][np.abs(y_predictions[:,-1]) < 0.5] = 0
y_predictions[:,-1][np.abs(y_predictions[:,-1]) > 0.5] = 1

loss_test = "{:10.4f}".format(mean_squared_error(y_test_all, y_predictions, squared=True))
metric_test = "{:10.4f}".format(mean_absolute_error(y_test_all, y_predictions))

loss_val = "{:10.4f}".format(mean_squared_error(y_val_all, y_predictions_val, squared=True))
metric_val = "{:10.4f}".format(mean_absolute_error(y_val_all, y_predictions_val))

loss_train = "{:10.4f}".format(mean_squared_error(y_train_all, y_predictions_train, squared=True))
metric_train = "{:10.4f}".format(mean_absolute_error(y_train_all, y_predictions_train))

print(metric_test, metric_val, metric_train)

In [ ]:
results_train = best_model.evaluate(train_dataset.map(lambda x, y1: (x, {'y1': y1})), verbose=1)
results_val = best_model.evaluate(val_dataset.map(lambda x, y1: (x, {'y1': y1})), verbose=1)
results_test = best_model.evaluate(test_dataset.map(lambda x, y1: (x, {'y1': y1,})), verbose=1)